### 함수 정의

In [15]:
import json
import requests
import pandas as pd

fitcloud_url = "https://aws-dev.fitcloud.co.kr"
corpId = "KDjAqAG0TnEAAAFK5eqDUL0A"

# account 별 월 별 사용량 (saving plan 포함)
# type: usage - ApplySavingsPlanCompute
def corp_month(
    start_month: str, 
    end_month: str,
    token: str,
    groupBy="account",
    ):
  api_url = fitcloud_url + "/service/trend/corp/month"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": start_month,
      "to": end_month,
      "groupBy": groupBy,
  }

  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # 일반 월 사용량에서 SavingPlan 가격을 제함
    return pd.DataFrame(resp.json())

  else:
    print("error")
#------------------------------------------------------------------------------------
# 월 입력값이  from: '201901', to: '202210'형태일 경우
# 시작 월부터 종료 월까지 리스트로 출력
from datetime import datetime, timedelta

def month_range(start_month, end_month):
    # Create datetime objects for start and end dates
    start_date = datetime.strptime(start_month, "%Y%m")
    end_date = datetime.strptime(end_month, "%Y%m")

    # Initialize list to store months
    months_list = []

    # Iterate over months and add them to the list
    current_month = start_date
    while current_month <= end_date:
        months_list.append(current_month.strftime("%Y%m"))
        current_month = (current_month + timedelta(days=32)).replace(day=1)

    return months_list
#-------------------------------------------------------------------------------------
# account 일자별 사용량을 반환
def ondemand_account_day(
    accountId: str, 
    day_from: str, 
    day_to: str, 
    token: str) -> float:
  api_url = fitcloud_url + "/ondemand/account/day"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": day_from,
      "to": day_to,
      "accountId": accountId,
  }
  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # JSON 형식으로 응답을 파싱 후 usageFee 합계를 구하기 위해 dataframe 의 변환
    df = pd.DataFrame(resp.json())
    usage_sum = round( df['usage_fee'].astype("Float32").sum(), 2)
    return usage_sum

  else:
    print("error")

def corp_month_internal(start_month: str, end_month: str, accountId: str, token: str):
  """calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.
  """
  json_data = corp_month(start_month, end_month, token)
  df = pd.DataFrame(json_data)
  print(df.columns)
  # accountId = accountId
  # account에 관련된 데이터 추출
  # df = df.query("accountId==@accountId")
  df = df[df['accountId'] == accountId]
  # 기간 내 월 리스트 추출
  month_list = month_range(start_month, end_month)
  # 월 column의 data type을 numeric으로 변환
  df_acc = df.copy()
  df_acc[month_list] = df_acc[month_list].apply(pd.to_numeric)
  # 내부 사용자용 filter: 합산에 포함시킬 항목
  internal_filter = ['Usage','ApplySavingsPlanCompute', 'ApplyRI' ]
  # internal_filter = ['Usage','ApplySavingsPlanCompute']
  df_int = df_acc.query("type in @internal_filter")
  sum = df_int[month_list].sum().sum()
  return sum

   

In [16]:
corp_month_internal('202309', '202309', '532805286864', "A7C0B830AA9EFE442BC6EFB056C297A9")

Index(['accountId', 'account_id', 'invoice_item', 'invoice_item_name',
       'lineItemType', '202309', 'invoiceItem', 'invoiceItemName',
       'line_item_type', 'type', 'currencyCode', 'currency_code'],
      dtype='object')


213.55291725389998

### 툴(Tool) 정의

In [17]:
# StructuredTool dataclass
# https://blog.langchain.dev/structured-tools/

from langchain.tools.base import StructuredTool

usage = StructuredTool.from_function(
    func=corp_month_internal,
    name="get_usage",
    description="calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars"
)

### LLM 정의 (추론엔진)

In [18]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.agents import AgentExecutor, create_structured_chat_agent

# Create bedrock runtime client
session = boto3.session.Session(region_name='us-east-1')
br_client = session.client("bedrock-runtime")

llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=br_client,
    model_kwargs={
        "temperature": 0,
    },
)

### Prompt 설정

In [19]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint

# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/structured-chat-agent")
prompt = hub.pull("richard-park/structured-chat-agent")
pprint(prompt)

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nstart_month and end_month value format is \'%Y%m\'.\nfor example, not \'2023-09\' but \'202309\'.\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final An

### Agent 설정

In [20]:
from langchain.agents import create_structured_chat_agent

tools = [ usage ]
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

### Run Agent

In [21]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
  agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
agent_executor.invoke({"input": "token 값은 ' '입니다. account id 532805286864의 2023년 9월에서 10월 자원 사용량은?"})